In [41]:
# statistics about attributes. Which attr. keys are there, which values?
# statistics about quality ratings -> quality depending on brand and other keywords


In [42]:
from os.path import join, basename, splitext
from glob import glob
from DLBio.helpers import MyDataFrame, load_json
import json

In [43]:
# READ ATTRIBUTES:

def to_dict(items_:list, file_id:str):
    out = {'Name':file_id}
    for item in items_:
        item = item.replace('\n', '')
        split = item.split(': ')
        key = split[0]
        value = ': '.join(split[1:])
        out[key] = value

    return out

def get_id(x:str):
    return splitext(basename(x))[0]

def read_attributes(to_dict, get_id):
    folder = 'guitar_attributes'
    df_attrib = MyDataFrame()
    for filename in glob(join(folder, '*.txt')):
        with open(filename, 'r') as filehandle:
            text = filehandle.readlines()
    
        df_attrib.update(
        to_dict(text, get_id(filename)),
        add_missing_values=True, missing_val=""
    )
    return df_attrib.get_df()

df_attrib = read_attributes(to_dict, get_id)
df_attrib.head(5)


,Name,Farbe,Bauart,Korpusform,Saitenanzahl,Halskonstruktion,Bundanzahl,Mensur,Korpusmaterial,Decke,...,Regler,Bridge/Tremolo,Hardware,Lieferumfang,Produktionsland,Serien-Nr.,Saitenstärke ab Werk,Besonderheit(en),Gewicht,Pickup Middle
0,d-angelico-premier-dc-fr,Fiesta Red,Semi-Hollow Body,ES-Modell,6-saitig,eingeleimt,22,"25"" (635 mm)","Ahorn, laminiert","Ahorn, laminiert",...,"2x Volume, 2x Tone",Tune-O-Matic mit Stop Tailpiece,Chrom,Gigbag,Südkorea,,,,,
1,prs-custom-24-wood-library-10132905,Fire Red To Grey Black Fade,Solid Body,PRS-Style,6-saitig,eingeleimt,24,"25"" (635 mm)",Mahagoni,"Ahorn, geriegelt (Figured Maple)",...,"1x Volume, 1x Tone",PRS Patented Tremolo,Nickel,Koffer + Zubehörset,USA,0321334,.010 - .046,Wood Library,"3,43 kg",
2,danelectro-dc59csb-12-string-hollowbody,Cherry Sunburst,Semi-Hollow Body,sonstige,12-saitig,verschraubt,21,,Masonite (Holzfaser),,...,"2x Volume, 2x Tone",feste Brücke,Chrom,,Südkorea,,,,,
3,squier-classic-vibe-60s-jazzmaster-3ts,3-Tone Sunburst,Solid Body,Offset,6-saitig,verschraubt,21,"25,5"" (648 mm)",Pappel (Poplar),,...,"1x Volume, 1x Tone",6-Saddle Vintage-Style with Non-Locking Floati...,Nickel,,Indonesien,,.009 - .042,Mustang Bridge,,
4,squier-classic-vibe-60s-competition-mustang-cpo,Capri Orange with Dakota Red Stripes,Solid Body,Offset,6-saitig,verschraubt,21,"24"" (610 mm)",Pappel (Poplar),,...,"1x Volume, 1x Tone",Floating Bridge with Dynamic Vibrato Tailpiece,Nickel,,Indonesien,,.009 - .042,Limited Edtion,,


In [57]:
len(df_attrib["Halskonstruktion"].unique())

3

In [49]:
# READ REVIEWS

def review_statistic(reviews_:list):
    tmp_df = MyDataFrame()
    for review in reviews_:
        tmp_df.update({k:v for k,v in review.items() if isinstance(v, float)})

    return tmp_df.get_df().mean(0).round(1)
    

folder = 'guitar_reviews'
df_rev = MyDataFrame()
for filename in glob(join(folder, '*.json')):
    data = load_json(filename)
    tmp = {
        'Name':get_id(filename),
        'num_reviews':len(data)
        }
    
    if tmp['num_reviews'] > 0:
        tmp.update(review_statistic(data))

    df_rev.update(tmp,
        add_missing_values=True, missing_val=0.
    )
    
df_rev = df_rev.get_df()

In [51]:
df_rev[df_rev.num_reviews > 0].sort_values("num_reviews", ascending=False).head(30)

,Name,num_reviews,Gesamteindruck,Ausstattung,Qualität,Preis/Leistung
560,brian-may-special-ac,26,5.0,4.8,4.8,4.9
2581,epiphone-les-paul-standard-50s-vintage-sunburst,20,4.9,5.0,4.8,4.8
1173,epiphone-les-paul-standard-60s-iced-tea,20,4.9,4.9,4.8,4.8
552,epiphone-les-paul-standard-60s-ebony,20,4.9,5.0,4.8,4.8
2594,ibanez-gio-grg170dx-bkn,17,4.7,4.5,4.5,4.8
272,slick-sl-60-tv,13,4.7,4.4,4.5,5.0
2176,ibanez-gio-grg170dx-bkn-10083991,12,4.9,4.7,4.6,5.0
843,slick-sl-59-bk,11,4.9,4.2,4.5,5.0
1952,slick-sl-50-sg,11,4.9,4.5,4.5,5.0
2421,ibanez-gio-grg121dx-wnf,10,4.5,4.2,4.5,4.5
